#### Link para download
- https://api-csvr.stg.cloud.cnj.jus.br/download_csv?tribunal=TJSP&indicador=&oj=&grau=&municipio=&ambiente=csv_p

#### FUNÇÕES APP

In [ ]:
import os
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from time import time, sleep

import zipfile
from tqdm import tqdm

import pandas as pd
import re
from collections import defaultdict

def download_with_retry(pasta_destino, nome_arquivo_zip, 
                        url="https://api-csvr.stg.cloud.cnj.jus.br/download_csv?tribunal=TJSP&indicador=&oj=&grau=&municipio=&ambiente=csv_p", 
                        max_retries=3, chunk_size=1024*1024):

    os.makedirs(pasta_destino, exist_ok=True)
    output_file = os.path.join(pasta_destino, nome_arquivo_zip)

    if os.path.exists(output_file):
        print(f"📦 Arquivo já existe: {output_file}")
        return output_file

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://justica-em-numeros.cnj.jus.br/"
    }

    retry_strategy = Retry(
        total=5,
        backoff_factor=5,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["GET"]
    )

    adapter = HTTPAdapter(max_retries=retry_strategy)

    for attempt in range(1, max_retries + 1):
        print(f"\n Tentativa {attempt} de {max_retries}...")
        try:
            with requests.Session() as session:
                session.mount("https://", adapter)
                session.mount("http://", adapter)

                with session.get(url, headers=headers, stream=True, timeout=180) as r:
                    r.raise_for_status()
                    total_size = int(r.headers.get("content-length", 0))
                    downloaded = 0
                    start = time()

                    with open(output_file, "wb") as f:
                        for chunk in r.iter_content(chunk_size=chunk_size):
                            if chunk:
                                f.write(chunk)
                                downloaded += len(chunk)
                                elapsed = time() - start
                                speed = downloaded / elapsed
                                percent = (downloaded / total_size) * 100
                                eta = (total_size - downloaded) / speed if speed else 0

                                print(
                                    f"\r{downloaded / (1024**2):.2f} MB "
                                    f"({percent:.2f}%) | Velocidade: {speed / (1024**2):.2f} MB/s | ETA: {eta:.1f}s",
                                    end=""
                                )

                    print("\n✅ Download concluído com sucesso.")
                    return output_file

        except Exception as e:
            print(f"\n❌ Erro: {e}")
            sleep(5)

    print("\n❌ Todas as tentativas de download falharam.")
    return None


In [ ]:
def descompactar_arquivos(zip_path, pasta_destino):
    print(f"Iniciando extração de: {zip_path}")
    os.makedirs(pasta_destino, exist_ok=True)

    inicio = time()
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        lista_arquivos = zip_ref.namelist()
        print(f"📁 Total de arquivos no ZIP: {len(lista_arquivos)}")

        with tqdm(total=len(lista_arquivos), desc="Extraindo", unit="arquivo") as pbar:
            for nome_arquivo in lista_arquivos:
                zip_ref.extract(nome_arquivo, pasta_destino)
                pbar.set_postfix_str(nome_arquivo[-60:])
                pbar.update(1)

    duracao = time() - inicio
    print(f"\nExtração concluída em {duracao:.2f} segundos.")
    print(f"📂 Arquivos extraídos em: {pasta_destino}")


In [ ]:
def agrupar_csvs(pasta_csvs, pasta_saida, chunksize=500_000):
    os.makedirs(pasta_saida, exist_ok=True)
    grupos = defaultdict(list)

    print("Varredura inicial dos arquivos...")
    for arquivo in os.listdir(pasta_csvs):
        if not arquivo.endswith(".csv"):
            continue
        caminho = os.path.join(pasta_csvs, arquivo)

        if arquivo.startswith("TJSP_tbl_correg"):
            print(f"📁 Detectado TJSP_tbl_correg: {arquivo}")
            inicio = time()
            df = pd.read_csv(caminho, sep=';', low_memory=False)
            duracao = time() - inicio
            print(f"Linhas: {len(df):,} | Tempo: {duracao:.2f}s")

            destino = os.path.join(pasta_saida, "TJSP_tbl_correg.csv")
            df.to_csv(destino, index=False, sep=';', encoding="utf-8-sig")
            print(f"✅ Salvo: {destino}")
            continue

        # Detecta chave por regex
        if re.search(r"TJSP_CN_.+\.csv$", arquivo):
            chave = "TJSP_CN"
        elif re.search(r"TJSP_CPL_15anos_.+\.csv$", arquivo):
            chave = "TJSP_CPL_15anos"
        elif re.search(r"TJSP_CPL_.+\.csv$", arquivo):
            chave = "TJSP_CPL"
        elif re.search(r"TJSP_Sent_.+\.csv$", arquivo):
            chave = "TJSP_Sent"
        elif re.search(r"TJSP_TBaix_.+\.csv$", arquivo):
            chave = "TJSP_TBaix"
        else:
            print(f"Ignorado (sem padrão): {arquivo}")
            continue

        grupos[chave].append(caminho)

    print("\nIniciando agrupamento por prefixo com chunks...\n")
    for grupo, arquivos in grupos.items():
        if not arquivos:
            continue

        destino = os.path.join(pasta_saida, f"{grupo}.csv")
        print(f"{grupo} → {len(arquivos)} arquivos")
        inicio_grupo = time()
        total_linhas = 0
        is_first_chunk = True

        for caminho in arquivos:
            print(f"Processando: {os.path.basename(caminho)}")
            t1 = time()

            for i, chunk in enumerate(pd.read_csv(caminho, sep=';', chunksize=chunksize, low_memory=False)):
                chunk.to_csv(destino, mode='w' if is_first_chunk else 'a', index=False, header=is_first_chunk, sep=';')
                is_first_chunk = False
                total_linhas += len(chunk)
                print(f"Chunk {i+1} → {len(chunk):,} linhas")

            t2 = time()
            print(f"Arquivo processado em {t2 - t1:.2f}s")

        duracao_total = time() - inicio_grupo
        print(f"✅ {grupo}.csv salvo | Total: {total_linhas:,} linhas | ⏱️ {duracao_total:.2f}s\n")
